### Create connection with azure datalake storage

### Steps
- Connection between Azure databricks and ADLS through Azure _**App registration**_
- Just register and app using that service
- Make a copy of appId and tenantId
- Through **certificate and secrets**, add a client secret
- Make a copy of secret_value
- Through **IAM**, add a role assignment
- Storage Blob Data Contributor
- Member should be the app created earlier

In [0]:
# Create the configs

CLIENT_ID:str = dbutils.secrets.get(scope="tokyo-olympic-secret-scope", key="client-id")
SECRET_ID:str = dbutils.secrets.get(scope="tokyo-olympic-secret-scope", key="secret-id")
TENANT_ID:str = dbutils.secrets.get(scope="tokyo-olympic-secret-scope", key="tenant-id")

configs = {
    "fs.azure.account.auth.type": "OAuth"
    ,"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider"
    ,"fs.azure.account.oauth2.client.id": CLIENT_ID
    ,"fs.azure.account.oauth2.client.secret": SECRET_ID
    ,"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/token"
}

In [0]:
# create the mounting point
# on the mounting source url, first part is the CONTAINER name
# second part is the STORAGE ACCOUNT name

mount_point = "/mnt/tokyo-olympic-data"

if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = "abfss://tokyo-olympic-data@mimran2021olympicdata.dfs.core.windows.net"
        ,mount_point=mount_point
        ,extra_configs = configs
    )

In [0]:
%fs
ls "mnt/tokyo-olympic-data/"

### Reading files form mounted location

In [0]:
from pyspark.sql.functions import col, round, format_number
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
# In databricks, we don't have to build spark session. Spark session gets auto create
# It can be checked by writing "spark"

# accessing csv  data from storage container
athletes = spark.read.format("csv").option("header","true").load("/mnt/tokyo-olympic-data/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").load("/mnt/tokyo-olympic-data/raw-data/coaches.csv")
gender_entries = spark.read.format("csv").option("header","true").load("/mnt/tokyo-olympic-data/raw-data/gender_entries.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo-olympic-data/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo-olympic-data/raw-data/teams.csv")

In [0]:
# Checking spark jobs
athletes.printSchema()
gender_entries.printSchema()

In [0]:
# Basic transformation
# transforming female column data into int data type
gender_entries = gender_entries.withColumn("Female", col("Female").cast(IntegerType()))\
    .withColumn("Male", col("Male").cast(IntegerType()))\
    .withColumn("Total", col("Total").cast(IntegerType()))

gender_entries.printSchema()

# InferSchema is used on medal and teams dataFrames
medals.printSchema()
teams.printSchema()
medals.show()

In [0]:
# Basic seletion and ordering
top_gold_medals = medals.orderBy("Total").select("Team/NOC","Total","Rank by Total")\
                    .show()

In [0]:
# calculating average number of entries by gender for each discipline
avg_by_gender = gender_entries.withColumn(
    "Avg_Female", format_number(col("Female")/ col("Total"),3)
    )\
    .withColumn(
        "Avg_Male", format_number(col("Male")/ col("Total"),3)
    )

avg_by_gender.show()

In [0]:
# exporting location -> /mnt/tokyo-olympic-data/transformed-data/
avg_by_gender.write.mode("overwrite")\
        .option("header","true")\
        .option("quoteAll","true")\
        .csv("/mnt/tokyo-olympic-data/transformed-data/avg_by_gender")

In [0]:
# exporting into a single file
athletes.repartition(1).write.mode("overwrite")\
        .option("header", "true")\
        .option("quoteAll", "true")\
        .csv("/mnt/tokyo-olympic-data/transformed-data/athletes")

coaches.repartition(1).write.mode("overwrite")\
        .option("header", "true")\
        .option("quoteAll","true")\
        .csv("/mnt/tokyo-olympic-data/transformed-data/coaches")

gender_entries.repartition(1).write.mode("overwrite")\
        .option("header", "true")\
        .option("quoteAll","true")\
        .csv("/mnt/tokyo-olympic-data/transformed-data/gender_entries")

medals.repartition(1).write.mode("overwrite")\
        .option("header","true")\
        .option("quoteAll","true")\
        .csv("/mnt/tokyo-olympic-data/transformed-data/medals")

teams.repartition(1).write.mode("overwrite")\
        .option("header","true")\
        .option("quoteAll","true")\
        .csv("/mnt/tokyo-olympic-data/transformed-data/teams")